# Website Summarizer

- Scrape
- Prompts
   - User/System

In [64]:
import requests
from bs4 import BeautifulSoup
from openai import OpenAI
from dotenv import load_dotenv
import os
from IPython.display import Markdown, display

## 1. Basic Scraper

In [29]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"}

class scraper:
    def __init__(self,url):
        self.url = url
        response = requests.get(url, headers = headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        for delet in soup.body(["script", "style", "img", "input"]):
            delet.decompose()
        self.body = soup.body
        self.text = soup.body.get_text(separator="\n", strip=True)
        self.title = soup.title.string if soup.title else "No Title"       

In [45]:
url = "https://edwarddonner.com"
url = "https://en.wikipedia.org/wiki/Tyrese_Haliburton"
webdata = scraper(url)
print(webdata.title)

Tyrese Haliburton - Wikipedia


## 2. Prompting
```python
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

In [46]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [47]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [48]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

## 3. GPT response

In [74]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
openai = OpenAI()

In [75]:
def summarize(url):
    website = scraper(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [76]:
summarize(url)

"# Tyrese Haliburton - Wikipedia Summary\n\nTyrese Haliburton is an American professional basketball player born on February 29, 2000, in Oshkosh, Wisconsin. He plays as a point guard for the Indiana Pacers in the NBA and is recognized as a two-time NBA All-Star and two-time All-NBA Third Team selection.\n\n## Early Life and High School Career\nHaliburton excelled in basketball during his time at Oshkosh North High School, where he was named Player of the Year in his conference and led his team to a state championship. During his junior and senior years, he averaged impressive stats and garnered various accolades.\n\n## College Career\nHaliburton played college basketball at Iowa State, where he broke multiple records, including assists in a single game. His sophomore season was cut short by an injury, yet he declared for the 2020 NBA Draft after an outstanding career.\n\n## Professional Career\n- **Sacramento Kings (2020–2022)**: Drafted 12th overall in the 2020 draft, Haliburton quic

In [77]:
def summarize_markdown(summary):
    return Markdown(summary)

In [78]:
summarize_markdown(summarize(url))

# Tyrese Haliburton - Wikipedia Summary

Tyrese Haliburton, born on February 29, 2000, in Oshkosh, Wisconsin, is an American professional basketball player for the Indiana Pacers in the NBA. He began his career with the Sacramento Kings after being drafted 12th overall in the 2020 NBA Draft. Haliburton has established himself as a standout player, earning two NBA All-Star selections and being named to the All-NBA Third Team twice as well.

## Early Life and Education
Haliburton grew up in Oshkosh and played high school basketball at Oshkosh North High School. He committed to Iowa State University for college basketball, where he made a significant impact early in his freshman year before suffering a season-ending wrist injury.

## Professional Career
- **Sacramento Kings (2020–2022)**: Haliburton had a successful rookie season, being named to the NBA All-Rookie First Team. He set multiple records and was recognized for his playmaking abilities.
  
- **Indiana Pacers (2022–present)**: Acquired in a trade with the Kings, Haliburton quickly became a core player for the Pacers. He achieved his first All-Star selection in 2023 and subsequently led the team to its first NBA Finals appearance in 2025. His recent seasons were marked by exceptional performance, including a notable triple-double and breaking various franchise records.

## National Team Career
Haliburton represented the United States at the FIBA Under-19 World Cup in 2019, where the team won gold. He also participated in the 2024 Summer Olympics, contributing to the team's success.

## Recent Highlights (2023-2025)
- **2023-24**: Named an All-Star and signed a significant contract extension.
- **2024-25**: Haliburton led the Pacers to the NBA Finals, becoming known for his clutch performances throughout the playoffs, though he faced an unfortunate Achilles tendon injury during the finals that could sideline him for an extended period.

## Personal Life
He is known for his strong faith and has expressed the importance of spirituality in his life. Haliburton is also related to several notable basketball players and is recognized for his high-volume shooting skills despite an unconventional shooting form.

This summary captures the essential aspects of Tyrese Haliburton’s life, career, and achievements, as detailed on his Wikipedia page.

## 4. Ollama response

In [69]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [81]:
messages = messages_for(scraper(url))

In [82]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [87]:
def ollama():
    response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
    response_content = response.json()['message']['content']
    return Markdown(response_content)
ollama()

Tyrese Haliburton is an American professional basketball player who plays as a point guard for the Indiana Pacers in the National Basketball Association (NBA). He was drafted by the Sacramento Kings with the 20th overall pick in the 2020 NBA draft and played for the team from 2020 to 2023.

Early Life and College Career
Haliburton was born on January 29, 2001, in Oshkosh, Wisconsin. He grew up playing basketball and soccer, and he attended Kettle Moraine Lutheran High School in Menomonee Falls, Wisconsin. Haliburton then played college basketball for Iowa State University from 2019 to 2020.

Professional Career
Haliburton was drafted by the Sacramento Kings with the 20th overall pick in the 2020 NBA draft. He made his professional debut in October 2020 and quickly became a key player for the team, averaging 14.6 points, 7.8 assists, and 4.3 rebounds per game.

In January 2022, Haliburton was traded to the Indiana Pacers along with Tyrese Maxey, Ben Simmons, and a draft pick in exchange for Domantas Sabonis, James Johnson, and a 2022 first-round pick. He quickly became a key player for the team, averaging 17.3 points, 8.5 assists, and 4.9 rebounds per game.

Awards and Accolades
Haliburton was named to the NBA All-Rookie Second Team in 2021 and was also selected as an NBA All-Star in 2023. He is considered one of the top young players in the league and is known for his exceptional passing ability, scoring versatility, and defensive skills.

International Career
Haliburton played for the United States men's national basketball team at the 2024 Summer Olympics, where he helped the team win a gold medal. He also competed in the FIBA Basketball World Cup in 2023, where the United States finished as runners-up.

Personal Life
Haliburton is a Christian and has spoken publicly about his faith. He is also known for his philanthropic work, particularly in his hometown of Oshkosh, Wisconsin.